In [1]:
# importing the module
import json
import re
import pandas as pd
import numpy as np
from IPython.display import display
from utils import *
from file_reader import *

In [4]:
def get_all_project_id(file_name):
    df = pd.read_csv(file_name, index_col=0)
    #display(df.index)
    return df.index
    

project_id_list = get_all_project_id('project_df_lib.csv')


Index(['8b4681b0-a16f-4a2d-9a86-6d4095fbbafe',
       '910c409c-bfbf-4a41-80e6-32b10edbc334',
       '067f8b2e-a07f-4725-bc07-c722469764b4',
       '269abf15-7013-4484-9846-1e7dbce6b0a6',
       '3150b827-f5fe-4b4b-8791-2a6da43d2d37',
       'de895b98-ebfd-4026-b6aa-cc976e8f61ff',
       'c351ca73-6904-406d-95dd-61b3ab60d8cf',
       '51b7cc4a-6ef6-403b-9e36-3a1e9e80df6c',
       '6da84e72-c3fc-43cb-a789-4a26b37fdb3f',
       'fc56cb69-6e1e-4aa8-9d6f-984357114f2d',
       ...
       '76a763a7-70fe-4def-8108-9b5cd7c5c7d7',
       'd9f2aaae-8b6a-4715-8680-ba8ba4881e7d',
       'cbd5a4a6-8366-4f51-a882-f9b173b5b214',
       '2091ba95-30d0-4797-b25e-11ce0de32a3a',
       '15b92527-eac0-4a52-90d8-3cbd0afa4547',
       '39a350b7-d591-46c9-bb30-2a87ab541750',
       '0f887772-4ff5-43ac-ac51-cd9567b7be14',
       '66ac5480-88d8-41a0-be1f-54b6b8d1b8da',
       '62c399e6-9cd3-4ff9-824c-afa9ff2f1db5',
       '5fbe15a8-c062-407f-9c7b-72b3ae4837cb'],
      dtype='object', length=238)

In [30]:

def get_lib_list(path,id_list,output_df):
    lib_list=[]
    df_header=['userID','itemID','rating']
    df = pd.DataFrame(columns=df_header)
    for i in id_list:
        libs = read_file(path,i)
        for j in libs:
            df2 = pd.DataFrame({'userID': [i],
                            'itemID': [j],
                            'rating': [1]})
            df = df.append(df2, ignore_index = True)

        lib_list = lib_list+libs
    #df.to_csv(output_df, index = False)
    #display(df)    
    lib_set = set(lib_list)
    return lib_set,df

def create_lib_df(path,id_list, output_df):
    libs,temp_df = get_lib_list(path,id_list,output_df)
    display(len(temp_df))
    list_of_zeroes = [0]*(len(libs)*len(id_list))
    index = pd.MultiIndex.from_product([id_list, libs], names = ["userID", "itemID"])
    df = pd.DataFrame(index = index)
    df['rating'] = list_of_zeroes
    for i in range(0, len(temp_df)):
        #print (temp_df.loc[i,'userID'])
        df.loc[temp_df.loc[i,'userID'],temp_df.loc[i,'itemID']]=1
    df.to_csv('surprise_libraries_df.csv')
    display(df)

create_lib_df('C:\\progetti\\low-code-ard-proj\\data-scraper\\code-snippets\\', project_id_list,output_df='temp_lib.csv')
#get_lib_list('C:\\progetti\\low-code-ard-proj\\data-scraper\\code-snippets\\',project_id_list)

544

rating
userID                               itemID                                
8b4681b0-a16f-4a2d-9a86-6d4095fbbafe #include <Arduino_LSM6DSOX.h>        0
                                     #include <FreqCount.h>               0
                                     #include "UK.h"                      0
                                     #include <RTCZero.h>                 0
                                     #include <TOTP.h>                    1
...                                                                     ...
5fbe15a8-c062-407f-9c7b-72b3ae4837cb #include <DFPlayerMini_Fast.h>       0
                                     #include <MKRGSM.h>                  0
                                     #include<mcp2515.h>                  0
                                     #include <AD5593R.h>                 0
                                     #include <SoftwareSerial.h>          0

[44268 rows x 1 columns]

In [31]:
from matplotlib.pyplot import get
from surprise import Dataset,accuracy, NormalPredictor, Reader,SVD, KNNWithMeans,KNNBasic, KNNWithZScore
from surprise.model_selection import cross_validate, train_test_split,KFold
import heapq
from collections import defaultdict
from operator import itemgetter


def precision_recall_at_k(predictions, k=10, threshold=0.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for  uid,_, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
    #print(user_est_true)
    #display(user_est_true)

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        #print('relevant: ',n_rel)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        #print('recommended: ',n_rec_k)

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls






df = pd.read_csv('temp_lib.csv')
 
#display(df)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0, 1))
#print(df.columns)
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[df.columns], reader)
algo = KNNWithMeans(sim_options={
        'name': 'msd',
        'user_based': False,
        })
        #'min_support':5,
        #'shrinkage':0

# Retrieve the trainset.
#trainset, testset = train_test_split(data, test_size=0.25)


# define a cross-validation iterator
kf = KFold(n_splits=2)


for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=0.2)
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))
    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)


# Train the algorithm on the trainset, and predict ratings for the testset
#algo.fit(trainset)
#predictions = algo.test(testset)
# We can now use this dataset as we please, e.g. calling cross_validate
#cross_validate(NormalPredictor(), data, cv=10)
#cross_validate(algo, data, measures=["RMSE", "MAE"], cv=10, verbose=True)



Computing the msd similarity matrix...
Done computing similarity matrix.
0.0
0.0
RMSE: 0.0882
Computing the msd similarity matrix...
Done computing similarity matrix.
0.0
0.0
RMSE: 0.0895


In [32]:
display (predictions)

[Prediction(uid='eb62195c-0fa7-4116-8ee6-1ba28b8190d4', iid='#include <Control_Surface.h>', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='df53ba2d-9afe-4797-a0d7-83e4bac4e5fa', iid='#include <LCDWIKI_GUI.h>', r_ui=0.0, est=0.029945629024092847, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='09c7d5c3-4f94-4248-9edf-1e3ab1037731', iid='#include "credentials.h"', r_ui=0.0, est=0.004992029420980461, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='557a8f51-aecd-487a-baaf-56f8b999faa2', iid='#include <TMRpcm.h>', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='d9f2aaae-8b6a-4715-8680-ba8ba4881e7d', iid='#include "bitmap.h"', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='2879c142-c263-457a-bf61-81e990821b60', iid='#include <MCP4131.h>', r_ui=0.0, est=0, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='7fd4feaf-ae53